# Imports

In [1]:
import numpy as np
import math
from scipy.stats import chi2
import os
from time import perf_counter_ns, perf_counter

In [2]:
from tqdm import tqdm
import torch
import timeit
import random

In [3]:
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [4]:
os.chdir("..")

In [5]:
from DistanceMetrics import CVM_Dist, Wasserstein_Dist, Anderson_Darling_Dist, Kolmogorov_Smirnov_Dist, DTS_Dist
from DistanceMetricsVec import CVM_Dist_p, Wasserstein_Dist_p, Anderson_Darling_Dist_p, Kolmogorov_Smirnov_Dist_p, DTS_Dist_p
from DistanceMetricsVec import ES_Dist

In [6]:
#The following library is developed by Fraunhofer and is still unreleased

from safeml.core.ecdf_distance_measures import AndersonDarlingDistance
from safeml.core.ecdf_distance_measures import KolmogorovSmirnovDistance
from safeml.core.ecdf_distance_measures import WassersteinDistance
from safeml.core.ecdf_distance_measures import CramerVonMisesDistance
from safeml.core.ecdf_distance_measures import DTSDistance

In [7]:
from safeml.DistanceMetrics import CVM_Dist_PVal
from safeml.DistanceMetrics import Anderson_Darling_Dist_PVal
from safeml.DistanceMetrics import Kolmogorov_Smirnov_Dist_PVal
from safeml.DistanceMetrics import Wasserstein_Dist_PVal
from safeml.DistanceMetrics import DTS_Dist_PVal

In [8]:
from DistanceMetricsVec import CVM_Dist_PVal as CVM_p
from DistanceMetricsVec import Anderson_Darling_Dist_PVal as AD_p
from DistanceMetricsVec import Kolmogorov_Smirnov_Dist_PVal as KS_p
from DistanceMetricsVec import Wasserstein_Dist_PVal as WS_p
from DistanceMetricsVec import DTS_Dist_PVal as DTS_p

# Generate ECDFs and Funcitons

In [9]:
#create random variables

XX1 = np.random.normal(1, 1, (10000, 1000))
YY1 = np.random.normal(3, 1, (10000, 1000))



In [10]:
CVM = CramerVonMisesDistance()
KS =  KolmogorovSmirnovDistance()
WS =  WassersteinDistance()
AD = AndersonDarlingDistance()
DTS =  DTSDistance()

Set up functions to test the distnace measures

In [11]:
def measure_original(XX, YY, method):
    length = len(XX)
    res = np.empty(length)
    for i in range(length):
        res[i] = method(XX[i], YY[i])
    return res

In [12]:
def measure_vec(XX, YY, method):
    length = len(XX)
    res = np.empty(length)
    for i in range(length):
        res[i] = method(XX[i], YY[i])
    return res

In [13]:
def measure_gpu(XX, YY, method):
    
    length = len(XX)
    xt = torch.tensor(np.array([yy.flatten() for yy in XX])).to("cuda")
    yt = torch.tensor(np.array([yy.flatten() for yy in YY])).to("cuda")
    
    res = torch.zeros(length)
    for i in range(length):
        res[i] = method._compute_distance_gpu(xt[i], yt[i], 1, 1, 1, "cuda")
    return res.numpy

In [14]:
def measure_es(XX, YY, method):
    length = len(XX)
    res = np.empty(length)
    for i in range(length):
        res[i], _ = method(XX[i], YY[i])
    return res

# Measure Normal Excution

### Cramer Von Mieses

In [15]:
%timeit measure_original(XX1, YY1, CVM_Dist)

18 s ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit measure_gpu(XX1, YY1, CVM)

7.24 s ± 58.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%timeit measure_vec(XX1, YY1, CVM_Dist_p)

1.57 s ± 5.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Anderson Darling

In [18]:
%timeit measure_original(XX1, YY1, Anderson_Darling_Dist)

26.6 s ± 159 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit measure_gpu(XX1, YY1, AD)

8.71 s ± 24.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit measure_vec(XX1, YY1, Anderson_Darling_Dist_p)

1.8 s ± 7.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Kolmogrov Smirnov

In [21]:
%timeit measure_original(XX1, YY1, Kolmogorov_Smirnov_Dist)

13.5 s ± 25.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit measure_gpu(XX1, YY1, KS)

7.05 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%timeit measure_vec(XX1, YY1, Kolmogorov_Smirnov_Dist_p)

1.55 s ± 6.77 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Wasserstein

In [24]:
%timeit measure_original(XX1, YY1, Wasserstein_Dist)

19.5 s ± 61 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit measure_gpu(XX1, YY1, WS)

5.86 s ± 65.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
%timeit measure_vec(XX1, YY1, Wasserstein_Dist_p)

1.54 s ± 8.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### DTS

In [27]:
%timeit measure_original(XX1, YY1, DTS_Dist)

28 s ± 54.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit measure_gpu(XX1, YY1, DTS)

8.91 s ± 71.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit measure_vec(XX1, YY1, DTS_Dist_p)

1.83 s ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ES

In [30]:
%timeit measure_es(XX1, YY1, ES_Dist)

5.68 s ± 53.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Measure P-Val Excution

In [31]:
#create new random vairable to test distnace with p-val

XX2 = np.random.normal(1, 1, (100, 1000))
YY2 = np.random.normal(3, 1, (100, 1000))

In [32]:
def measure_pval(XX, YY, length, method):
    res = np.zeros(length)
    for i in range(length):
        _, res[i] = method(XX[i], YY[i])

    return res

In [33]:
def measure_pval_gpu(XX, YY, length, method):
    res = np.zeros(length)
    xt = torch.tensor(np.array([yy.flatten() for yy in XX])).to("cuda")
    yt = torch.tensor(np.array([yy.flatten() for yy in YY])).to("cuda")
    
    for i in range(length):
        res[i], _ = method.measure_metric_p_value_gpu(XX[i], YY[i])

    return res

### Cramer von Mieses

In [34]:
%timeit measure_pval(XX2, YY2, 100,CVM_Dist_PVal)

4min 13s ± 245 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit measure_pval_gpu(XX2, YY2, 100, CVM)

3min 42s ± 1.62 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
%timeit measure_pval(XX2, YY2, 100, CVM_p)

23.9 s ± 129 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Anderson Darling

In [37]:
%timeit measure_pval(XX2, YY2, 100, Anderson_Darling_Dist_PVal)

5min 35s ± 640 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
%timeit measure_pval_gpu(XX2, YY2, 100, AD)

4min 8s ± 247 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%timeit measure_pval(XX2, YY2, 100, AD_p)

27.5 s ± 23.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Kolmogrov Smirnov

In [40]:
%timeit measure_pval(XX2, YY2, 100, Kolmogorov_Smirnov_Dist_PVal)

3min 36s ± 139 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
%timeit measure_pval_gpu(XX2, YY2, 100, KS)

3min 54s ± 772 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%timeit measure_pval(XX2, YY2, 100, KS_p)

23.4 s ± 94.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Wasserstein

In [43]:
%timeit measure_pval(XX2, YY2, 100, Wasserstein_Dist_PVal)

4min 43s ± 277 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%timeit measure_pval_gpu(XX2, YY2, 100, WS)

3min 33s ± 206 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%timeit measure_pval(XX2, YY2, 100, WS_p)

23.3 s ± 33.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### DTS

In [46]:
%timeit measure_pval(XX2, YY2, 100, DTS_Dist_PVal)

6min 15s ± 285 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
%timeit measure_pval_gpu(XX2, YY2, 100, DTS)

4min 14s ± 1.31 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%timeit measure_pval(XX2, YY2, 100, DTS_p)

26.3 s ± 39.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ES

In [49]:
%timeit measure_pval(XX2, YY2, 100, ES_Dist)

57.5 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
